In [126]:
import mysql.connector
from mysql.connector.constants import ClientFlag
import pandas as pd
import numpy as np

file = open("../Data/cloudsqlpassword.txt", "r")
pwd = file.read()

config = {
    'user': 'root',
    'password': pwd,
    'host': '35.199.36.223',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': '../Data/ssl/server-ca.pem',
    'ssl_cert': '../Data/ssl/client-cert.pem',
    'ssl_key': '../Data/ssl/client-key.pem'
}

# now we establish our connection
cnxn = mysql.connector.connect(**config)

In [127]:
config['database'] = 'bikedata'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

In [128]:
# get dataframe
data = pd.read_csv('../Data/HealthyRideRentals2015Q2.csv')

data = data.rename(columns={"Trip id": "Trip_id", "From station id": "From_station_id", "From station name": "From_station_name",
                    "To station id": "To_station_id", "To station name": "To_station_name"})
#data = data.astype({'Trip_id':np.int32,'Bikeid':np.int32,'Tripduration':np.int32,'From_station_id':np.int32,'To_station_id':np.int32})

data.dtypes

Trip_id               int64
Starttime            object
Stoptime             object
Bikeid                int64
Tripduration          int64
From_station_id       int64
From_station_name    object
To_station_id         int64
To_station_name      object
Usertype             object
dtype: object

In [129]:
# DROP TABLE
sql = "DROP TABLE ride_rental_data"

cursor.execute(sql)


#CREATE TABLE
cursor.execute("CREATE TABLE ride_rental_data ("
               "Trip_id INT(64),"
               "Starttime DATETIME,"
               "Stoptime DATETIME,"
               "Bikeid INT(64),"
               "Tripduration INT(64),"
               "From_station_id INT(64),"
               "From_station_name VARCHAR(255),"
               "To_station_id INT(64)," 
               "To_station_name VARCHAR(255),"
               "Usertype VARCHAR(255) )")

cnxn.commit()  # this commits changes to the database

In [132]:
# first we setup our query
query = ("INSERT INTO ride_rental_data (Trip_id, Starttime, Stoptime, Bikeid, Tripduration, From_station_id, From_station_name, To_station_id, To_station_name, Usertype) "
         "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

# then we execute with every row in our dataframe
#cursor.executemany(query, list(data.to_records(index=False)))
cursor.executemany(query, list(data))
cnxn.commit()  # and commit changes

ProgrammingError: Not enough parameters for the SQL statement

In [ ]:
list(data.to_records(index=False))

In [ ]:
data